In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://ftp.itu.edu.tr/Mirror/Apache/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar xf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

!ls

from pyspark import SparkContext
sc = SparkContext.getOrCreate()

lines = sc.textFile("sample_data") 
lines.take(5)
#number of lines
print(lines.count())



PyGame-Learning-Environment  spark-2.4.0-bin-hadoop2.7
sample_data		     spark-2.4.0-bin-hadoop2.7.tgz
50068


In [2]:
import urllib.request
import pandas as pd 
import zipfile
import os
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

import os

small=urllib.request.urlretrieve('http://files.grouplens.org/datasets/movielens/ml-latest-small.zip','/content/drive/My Drive/small.zip')
big=urllib.request.urlretrieve('http://files.grouplens.org/datasets/movielens/ml-latest.zip','/content/drive/My Drive/big.zip')

with zipfile.ZipFile('/content/drive/My Drive/small.zip', "r") as z:
   z.extractall('/content/drive/My Drive/')
    
with zipfile.ZipFile('/content/drive/My Drive/big.zip', "r") as z:
   z.extractall('/content/drive/My Drive/')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!ls "/content/drive/My Drive"

 big.zip  'Colab Notebooks'   ml-latest   ml-latest-small   small.zip


In [0]:
small_ratings_raw_data = sc.textFile('/content/drive/My Drive/ml-latest-small/ratings.csv')
small_ratings_raw_data_header = small_ratings_raw_data.take(1)[0]


In [0]:
small_ratings_data = small_ratings_raw_data.filter(lambda line: line!=small_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()

In [7]:
small_ratings_data.take(3)

[('1', '1', '4.0'), ('1', '3', '4.0'), ('1', '6', '4.0')]

In [8]:
small_movies_raw_data = sc.textFile('/content/drive/My Drive/ml-latest-small/movies.csv')
small_movies_raw_data_header = small_movies_raw_data.take(1)[0]

small_movies_data = small_movies_raw_data.filter(lambda line: line!=small_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1])).cache()
    
small_movies_data.take(3)

[('1', 'Toy Story (1995)'),
 ('2', 'Jumanji (1995)'),
 ('3', 'Grumpier Old Men (1995)')]

In [0]:
training_RDD, validation_RDD, test_RDD = small_ratings_data.randomSplit([0.6, 0.2, 0.2], seed=42)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))


In [40]:
from pyspark.mllib.recommendation import ALS
import math

seed = 42
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    model = ALS.train(training_RDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print('For rank %s the RMSE is %s'+ str( (rank, error)))
    if error < min_error:
        min_error = error
        best_rank = rank

print('The best model was trained with rank %s'+ str( best_rank))

For rank %s the RMSE is %s(4, 0.9085735126459838)
For rank %s the RMSE is %s(8, 0.9093646061880156)
For rank %s the RMSE is %s(12, 0.9062301308911616)
The best model was trained with rank %s12


In [41]:
predictions.take(3)

[((140, 1084), 3.5153012095701484),
 ((74, 1084), 4.177394484005578),
 ((402, 1084), 3.9674954778809974)]